In [1]:
import pandas as pd
import os

In [2]:
data_folder = "data/synpop_related"
marginals = pd.read_csv(os.path.join(data_folder, "hh_marginals_ipu.csv"), header=[0,1])
marginals = marginals.set_index(marginals.columns[0])
marginals = marginals.drop(columns=marginals.columns[0])
totals = marginals.sum(axis=1)/5 #n_atts = 5

In [3]:
pool = pd.read_csv(os.path.join(data_folder, "HH_pool.csv"))

In [4]:
# marginals.columns = marginals.columns.to_flat_index()
chosen_att = "hhsize"
filtered_marginal = marginals[marginals.columns[marginals.columns.get_level_values(0) == chosen_att]]
states = set(filtered_marginal.columns.get_level_values(1))
filtered_pools = {state: pool[pool[chosen_att]==state] for state in states}

In [5]:
have_enough_neg_income = False
synthetic_population = None
while not have_enough_neg_income:
    store_pop = []
    for zone, row in filtered_marginal.iterrows():
        for state in states:
            pop_count = row[(chosen_att, state)]
            if pop_count > 0:
                state_pool = filtered_pools[state]
                pop = state_pool.sample(int(pop_count), replace=True)
                store_pop.append(pop)
    synthetic_population = pd.concat(store_pop)
    check = synthetic_population[synthetic_population["hhinc"]=="Negative income"]
    if len(check) > 1:
        have_enough_neg_income = True
        print("Have enough negative income households")

Have enough negative income households


In [6]:
# store_pop = []
# for zone, tot in zip(totals.index, totals):
#     syn_pop = pool.sample(n=int(tot))
#     syn_pop["zone_id"] = zone
#     store_pop.append(syn_pop)
# synthetic_population = pd.concat(store_pop)

In [7]:
synthetic_population.to_csv("sampled_BN_syn.csv", index=False)